In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
df = pd.read_csv('D:\\Users\\flammmes\\Downloads\\Book1.csv')
df['d']=df['d'].str.replace(',','')
dfn = pd.read_csv('D:\\Users\\flammmes\\Downloads\\Fluids data.csv')


df.dtypes
dd = df.d.astype('float')
df.d = df.d.astype('float')
df.drop(['Rsob','STO'],axis=1,inplace=True)

x = df.iloc[:,1]/df.iloc[:,8]
mu = 1000*(df.iloc[:,3]*x*x+df.iloc[:,4]*x+df.iloc[:,5])/(df.iloc[:,6]*x+1)
df.insert(9,'μob',mu)
df = df[df['μob']<=50] 

df["μob_cat"] = pd.cut(df["μob"],
                               bins=[0., 5, np.inf],
                               labels=[1, 2])
from sklearn.model_selection import StratifiedShuffleSplit
c=[]
for i in range(100):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    for train_index, test_index in split.split(df, df["μob_cat"]):
        strat_train_set = df.iloc[train_index]
        strat_test_set = df.iloc[test_index]

    for set_ in (strat_train_set, strat_test_set):
        set_.drop("μob_cat", axis=1, inplace=True)
        #set_.drop("μob", axis=1, inplace=True)
    df1 = strat_train_set.copy()
    df2 = strat_test_set.copy()
    df1["μob_cat"] = pd.cut(df1["μob"],
                                   bins=[0., 5, np.inf],
                                   labels=[1, 2])
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    for ti, tei in split.split(df1, df1["μob_cat"]):
        strat_tr_set = df1.iloc[ti]
        strat_val_set = df1.iloc[tei]
    for set_ in (strat_tr_set, strat_val_set):
        set_.drop("μob_cat", axis=1, inplace=True)
        #set_.drop("μob", axis=1, inplace=True)
    df_train = strat_tr_set.copy()
    df_val = strat_val_set.copy()


    def gen_func(a):

        dflist = []
        for i in range(len(a)):

            correlations_select = pd.DataFrame({'P':P_i,'μ':mu_i})
            pressure_bubble_point = correlations_select.iloc[0,0]*np.linspace(1,1,20)
            visc_bubble_point = correlations_select.iloc[0,1]*np.linspace(1,1,20)
            correlations_select.insert(0,"Pb",pressure_bubble_point)
            correlations_select.insert(2,'μob',visc_bubble_point)
            dflist.append(correlations_select)


        totaldf = pd.concat(dflist)
        totaldf.Pb = totaldf.Pb*145.038
        totaldf.P = totaldf.P*145.038
        totaldf.insert(2,'ΔP', totaldf.P-totaldf.Pb)
    #totaldf.drop('P',axis=1,inplace=True)
        return totaldf
        
        #totaldf.API = 141.5*1000/totaldf.API -131.5
        #totaldf.Rsob = totaldf.Rsob*5.614583333333333
        #totaldf = totaldf[totaldf.μ != totaldf.μob]
        #totaldf.drop('P',axis=1,inplace=True)
        return totaldf
    test_data = gen_func(df2)
    train_data = gen_func(df_train)
    val_data = gen_func(df_val)

    #y = test_data[test_data.ΔP==0]
    #x =  val_data[val_data.ΔP==0]
    #train_data = pd.concat([train_data,y,x])
    all_data = gen_func(df)
    for a in [test_data,train_data,val_data,all_data]:
        a.μ = np.sqrt(np.log(a.μ/a.μob+0.0012))
        a.ΔP = np.power(a.ΔP,0.5)
        #a.ΔP = np.log(a.ΔP+1)
        #a.insert(0,'x1',a.ΔP*np.power(a.P/a.Pb,0.2))
        a.μob = a.ΔP*np.power(a.μob,0.07)

        #a.insert(4,'logμob',a.μob*np.log(a.μob))
        a.Pb = a.ΔP/np.log(a.Pb+400)
        a.P = a.ΔP*np.power(a.P,-0.15)
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import PolynomialFeatures

    X_train = train_data.drop(["μ"], axis=1) 
    y_train = train_data["μ"].copy()
    X_val = val_data.drop(["μ"], axis=1) 
    y_val = val_data["μ"].copy()
    X_test = test_data.drop(["μ"], axis=1) # drop labels for training set
    y_test = test_data["μ"].copy()

    poly_features = PolynomialFeatures(degree=2, include_bias=False)
    X_train_poly = poly_features.fit_transform(X_train)
    X_val_poly = poly_features.fit_transform(X_val)
    X_test_poly = poly_features.fit_transform(X_test)
    poly_scaler = StandardScaler()
    X_train_poly = poly_scaler.fit_transform(X_train_poly)
    X_val_poly = poly_scaler.transform(X_val_poly)
    X_test_poly = poly_scaler.transform(X_test_poly)

    scaler = StandardScaler()
    X_train= scaler.fit_transform(X_train)
    X_val= scaler.transform(X_val)
    X_test= scaler.transform(X_test)
    X_features_full = [X_train,X_val,X_test]
    X_features_poly_full = [X_train_poly,X_val_poly,X_test_poly]
    y_labels_full = [y_train,y_val,y_test]
    totaldf_full = gen_func(df2)
    all_data_full = gen_func(df)

    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_percentage_error
    from sklearn.svm import SVR
    X_features = X_features_full
    X_features_poly = X_features_poly_full
    y_labels = y_labels_full
    from sklearn.linear_model import Ridge
    ridge_reg = Ridge(alpha=0.2, solver="cholesky")
    ridge_reg.fit(X_features_poly[0], y_labels[0])
    svm_rbf_reg = SVR(kernel="rbf",C=1,epsilon=0.001)
    svm_rbf_reg.fit(X_features[0], y_labels[0])
    from sklearn.ensemble import RandomForestRegressor

    forest_reg = RandomForestRegressor(n_estimators=100)
    forest_reg.fit(X_features[0], y_labels[0])
    from sklearn.ensemble import VotingRegressor
    voting_reg = VotingRegressor(
        estimators=[('fr', forest_reg), ('svm', svm_rbf_reg)])
    voting_reg.fit(X_features[0],y_labels[0])
    import tensorflow as tf
    from tensorflow import keras
    model = keras.models.Sequential([
        keras.layers.Dense(500, activation="relu", input_shape=X_features[0].shape[1:]),
        keras.layers.Dense(1)
    ])
    for j in range(2):
        model.compile(loss="mean_absolute_percentage_error", optimizer=keras.optimizers.Adam())
        history = model.fit(X_features[0], y_labels[0], epochs=20,batch_size=16, validation_data=(X_features[1], y_labels[1]))
    totaldf = totaldf_full
    totaldf.insert(5,'poly ridge μ', totaldf.μob*(np.exp(np.power(ridge_reg.predict(X_features_poly[2]),2)) -0.0012))
    totaldf.insert(6,'rbf μ', totaldf.μob*(np.exp(np.power(svm_rbf_reg.predict(X_features[2]),2)) -0.0012))
    totaldf.insert(7,'forest μ', totaldf.μob*(np.exp(np.power(forest_reg.predict(X_features[2]),2)) -0.0012))
    totaldf.insert(8,'voting μ', totaldf.μob*(np.exp(np.power(voting_reg.predict(X_features[2]),2)) -0.0012))
    totaldf.insert(9,'sequential', (np.exp(np.power(model.predict(X_features[2]),2)) -0.0012))
    totaldf.sequential = totaldf.sequential*totaldf.μob
    totaldf.insert(10,'Beal', totaldf['μob'] +(0.001*totaldf['ΔP'])*(0.024*np.power(totaldf['μob'] ,1.6)+0.038*np.power(totaldf['μob'] ,0.56)))
    totaldf.insert(11,'Kouzel modified',totaldf['μob']*np.exp((-2.34864*np.power(10,-5.0)+9.30705*np.power(10,-5.0)*np.power(totaldf['μob'],0.181))*totaldf.ΔP))
    def Stats_matrix(a):
        absolute_average_error = []
        for i in range(7):
            absolute_average_error.append(np.average(np.absolute((a.iloc[:,i+5]-a.μ)/a.μ*100)))
        statistics  = pd.Series(list(absolute_average_error))
        return statistics

    Stats_matrix(totaldf)

    y = Stats_matrix(totaldf).values.tolist()
    c.append(y)
    

C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Epoch 1/20
455/455 [==============================] - 1s 2ms/step - loss: 27.3392 - val_loss: 28.7160
Epoch 2/20
455/455 [==============================] - 1s 1ms/step - loss: 22.2036 - val_loss: 17.4354
Epoch 3/20
455/455 [==============================] - 1s 1ms/step - loss: 20.2385 - val_loss: 53.9492
Epoch 4/20
455/455 [==============================] - 1s 2ms/step - loss: 21.3909 - val_loss: 15.2374
Epoch 5/20
455/455 [==============================] - 1s 1ms/step - loss: 19.4740 - val_loss: 16.0226
Epoch 6/20
455/455 [==============================] - 1s 1ms/step - loss: 18.9474 - val_loss: 16.5198
Epoch 7/20
455/455 [==============================] - 1s 2ms/step - loss: 18.0570 - val_loss: 14.0971
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 17.9453 - val_loss: 18.3744
Epoch 9/20
455/455 [==============================] - 1s 2ms/step - loss: 18.0573 - val_loss: 14.8959
Epoch 10/20
455/455 [==============================] - 1s 1ms/step - loss: 16.8073

C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Epoch 1/20
455/455 [==============================] - 1s 2ms/step - loss: 25.7219 - val_loss: 47.4271
Epoch 2/20
455/455 [==============================] - 1s 2ms/step - loss: 23.7949 - val_loss: 18.1182
Epoch 3/20
455/455 [==============================] - 1s 1ms/step - loss: 23.3556 - val_loss: 16.9384
Epoch 4/20
455/455 [==============================] - 1s 2ms/step - loss: 20.3780 - val_loss: 18.4022
Epoch 5/20
455/455 [==============================] - 1s 2ms/step - loss: 19.7344 - val_loss: 19.6940
Epoch 6/20
455/455 [==============================] - 1s 2ms/step - loss: 20.7920 - val_loss: 29.5643
Epoch 7/20
455/455 [==============================] - 1s 2ms/step - loss: 17.9641 - val_loss: 32.9992
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 19.0478 - val_loss: 17.2108
Epoch 9/20
455/455 [==============================] - 1s 2ms/step - loss: 18.7090 - val_loss: 22.9633
Epoch 10/20
455/455 [==============================] - 1s 2ms/step - loss: 17.5597

C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


...

Epoch 1/20
455/455 [==============================] - 1s 2ms/step - loss: 26.7490 - val_loss: 21.2311
Epoch 2/20
455/455 [==============================] - 1s 2ms/step - loss: 23.5433 - val_loss: 17.7981
Epoch 3/20
455/455 [==============================] - 1s 2ms/step - loss: 22.0340 - val_loss: 17.9258
Epoch 4/20
455/455 [==============================] - 1s 2ms/step - loss: 23.6274 - val_loss: 17.5832
Epoch 5/20
455/455 [==============================] - 1s 2ms/step - loss: 19.3731 - val_loss: 19.0671
Epoch 6/20
455/455 [==============================] - 1s 2ms/step - loss: 20.1344 - val_loss: 14.2037
Epoch 7/20
455/455 [==============================] - 1s 2ms/step - loss: 19.1453 - val_loss: 15.2788
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 18.1957 - val_loss: 15.4804
Epoch 9/20
455/455 [==============================] - 1s 2ms/step - loss: 17.5124 - val_loss: 15.8993
Epoch 10/20
455/455 [==============================] - 1s 2ms/step - loss: 17.7833

C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Epoch 1/20
455/455 [==============================] - 2s 3ms/step - loss: 26.6192 - val_loss: 18.2033
Epoch 2/20
455/455 [==============================] - 1s 3ms/step - loss: 24.2754 - val_loss: 20.8141
Epoch 3/20
455/455 [==============================] - 1s 2ms/step - loss: 21.1709 - val_loss: 21.1786
Epoch 4/20
455/455 [==============================] - 1s 2ms/step - loss: 21.9189 - val_loss: 16.1680
Epoch 5/20
455/455 [==============================] - 1s 2ms/step - loss: 20.6737 - val_loss: 14.0035
Epoch 6/20
455/455 [==============================] - 1s 2ms/step - loss: 18.9465 - val_loss: 16.4896
Epoch 7/20
455/455 [==============================] - 1s 2ms/step - loss: 18.4089 - val_loss: 12.1570
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 18.7251 - val_loss: 13.2862
Epoch 9/20
455/455 [==============================] - 1s 2ms/step - loss: 17.7123 - val_loss: 13.8773
Epoch 10/20
455/455 [==============================] - 1s 2ms/step - loss: 17.6127

C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Epoch 1/20
455/455 [==============================] - 1s 2ms/step - loss: 26.0803 - val_loss: 20.8647
Epoch 2/20
455/455 [==============================] - 1s 2ms/step - loss: 22.9445 - val_loss: 14.7568
Epoch 3/20
455/455 [==============================] - 1s 2ms/step - loss: 20.8639 - val_loss: 17.0801
Epoch 4/20
455/455 [==============================] - 1s 2ms/step - loss: 24.0249 - val_loss: 22.6316
Epoch 5/20
455/455 [==============================] - 1s 2ms/step - loss: 19.8333 - val_loss: 20.6271
Epoch 6/20
455/455 [==============================] - 1s 2ms/step - loss: 19.5400 - val_loss: 13.7327
Epoch 7/20
455/455 [==============================] - 1s 2ms/step - loss: 17.9161 - val_loss: 13.0461
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 19.3630 - val_loss: 19.3299
Epoch 9/20
455/455 [==============================] - 1s 1ms/step - loss: 17.8393 - val_loss: 14.6528
Epoch 10/20
455/455 [==============================] - 1s 1ms/step - loss: 17.9482

In [2]:
c

[[3.511067573948755,
  3.4572880710886493,
  3.4683837303132075,
  3.4005526446996255,
  3.424680999250948,
  3.5409292769533436,
  4.08539172201245],
 [3.7129746418229668,
  3.6026754305062796,
  3.68305202848489,
  3.5499935096545965,
  3.5740842818368384,
  3.6961352946131205,
  4.299340085372981],
 [3.297609318127598,
  3.1491799557560594,
  3.3038075158372133,
  3.1324716234593404,
  3.074597070714341,
  3.2017284818284923,
  3.65547398612894],
 [3.04014412481711,
  2.974443240982125,
  2.930589067238445,
  2.845381251708104,
  2.929118819002436,
  2.999185845055297,
  3.503234230527983],
 [3.4410044948759797,
  3.342457212875361,
  3.4032950976639467,
  3.242895423499005,
  3.3395337534469562,
  3.5165036952881032,
  3.9336670928241304],
 [3.3866207964062256,
  3.351901765918905,
  3.465809800429322,
  3.2878888093421663,
  3.429574097196419,
  3.3467782986813437,
  3.9626032674858274],
 [4.070800608747023,
  3.9996906178251423,
  4.18303798060739,
  4.026270092123165,
  3.996597

In [11]:
f1 = pd.DataFrame(c)
f1
%store f1

Stored 'f1' (DataFrame)
